<a href="https://colab.research.google.com/github/wayneotemah/Eng2Sheng-notebooks/blob/main/Sheng_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sheng Web scraper 🚀🌐🤖

In [ ]:
import requests
from bs4 import BeautifulSoup

## Single page scrape

First we will perform a single page scrape on teh target site to see how the data is...


We will be targeting the site: https://shengilia.blogspot.com/

This site has all the verses of first 5 books of the bible writen New Tersterment written in English and the sheng text


All the Enlsish text are in the `<br>` tag, with the sheng transaltion text is in the `<i>`

In [ ]:
url = 'https://shengilia.blogspot.com/'
response = requests.get(url)


In [ ]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the text inside <b> and <i> tags
english_texts = [tag.get_text(strip=True) for tag in soup.find_all('b')]
sheng_texts = [tag.get_text(strip=True) for tag in soup.find_all('i')]

### Examine the data

In [ ]:
english_texts[:10]

In [ ]:
sheng_texts[:10]

The  data in the Enlish text is not in the format we want

1. There are mullitple unwanted blank items in the  list

Solution

1. Remove them


### English text processing

In [ ]:
pattern = r'[0-9]'

In [ ]:
#remove blank items in the list
for item in english_texts:
  if item == '':
    english_texts.remove(item)

### Combine the texts

In [ ]:
#Combine the extracted texts
translations = list(zip(english_texts[1:], sheng_texts))

# Print the extracted translations
for english, sheng in translations:
    print(f"English: {english}\nSheng: {sheng}\n")

In [ ]:
# save translations in pandas dataframe
import pandas as pd
df = pd.DataFrame(translations, columns=['English', 'Sheng'])
# save the dataframe to an CSV file
df.to_csv('translations.csv', index=False)

## Full site extraction

###Extract  target urls

In [ ]:
archive_list = soup.find(id="BlogArchive1_ArchiveList")

In [ ]:
main_page_urls = [a['href'] for a in archive_list.find_all('a', href=True)]
main_page_urls

['javascript:void(0)',
 'https://shengilia.blogspot.com/2010/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2010/10/',
 'https://shengilia.blogspot.com/2010/10/acts-91-15.html',
 'https://shengilia.blogspot.com/2010/10/acts-916-30.html',
 'https://shengilia.blogspot.com/2010/10/acts-931-43.html',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2010/09/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2010/08/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/12/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/11/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/10/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/09/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/08/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/07/',
 'javascript:void(0)',
 'https://shengilia.blogspot.com/2009/06/']

In [ ]:
#remove all items endong with html and javascript
main_page_urls = [url for url in main_page_urls if not url.endswith(('.html'))]
main_page_urls = [url for url in main_page_urls if not "javascript" in url ]
main_page_urls

['https://shengilia.blogspot.com/2010/',
 'https://shengilia.blogspot.com/2010/10/',
 'https://shengilia.blogspot.com/2010/09/',
 'https://shengilia.blogspot.com/2010/08/',
 'https://shengilia.blogspot.com/2009/',
 'https://shengilia.blogspot.com/2009/12/',
 'https://shengilia.blogspot.com/2009/11/',
 'https://shengilia.blogspot.com/2009/10/',
 'https://shengilia.blogspot.com/2009/09/',
 'https://shengilia.blogspot.com/2009/08/',
 'https://shengilia.blogspot.com/2009/07/',
 'https://shengilia.blogspot.com/2009/06/']

In [ ]:
#using the main_page_urls, lets ectract all the urls which ends with html
target_urls = []
for url in main_page_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    archive_list = soup.find(id="BlogArchive1_ArchiveList")
    urls = [a['href'] for a in archive_list.find_all('a', href=True)]

    #only add urls with html in the string
    for url in urls:
        if url.endswith('.html'):
          if "ma-thanks" in url:
            pass
          else:
            target_urls.append(url)


In [ ]:
target_urls

##  Use target urls for extraction

In [ ]:
def extract_translations(url):
  soup = BeautifulSoup(response.content, 'html.parser')

  english_texts = [tag.get_text(strip=True) for tag in soup.find_all('b')]
  sheng_texts = [tag.get_text(strip=True) for tag in soup.find_all('i')]

  #remove blank items in the list
  for item in english_texts:
    if item == '':
      english_texts.remove(item)

  #Combine the extracted texts
  translations = list(zip(english_texts[1:], sheng_texts))
  return translations

In [ ]:
final_translations = []
for url in target_urls:
  response = requests.get(url)
  translations = extract_translations(response)
  for english, sheng in translations:
    print(f"English: {english}\nSheng: {sheng}\n")
  final_translations.extend(translations)



In [ ]:
final_translations

[("1Meanwhile, Saul was still breathing out murderous threats against the Lord's disciples. He went to the high priest",
  'So niaje aje, Paulo alikuwa bado akispit vitisho za kumada ma disciple. Akaishia kwa kuhani mumeja'),
 ('2and asked him for letters to the synagogues in Damascus, so that if he found any there who belonged to the Way, whether men or women, he might take them as prisoners to',
  'Akamshow niasaje, amgei ma letter kwa zile synagogue za majuu, huko Damascus, njo ibidi, akisaka mse any alibelong kwa wokovu, either manzi au boi, ikuwe ni jela hadi'),
 ('Jerusalem.', 'Jerusa.'),
 ('3As he neared Damascus on his journey, suddenly a light from heaven flashed around him.',
  'Akikaribia kwa safari, akiona Damascus na far,pap! ki light kikatokea huratiti ikaflash imemsurround.'),
 ('4He fell to the ground and heard a voice say to him, "Saul, Saul, why do you persecute me?"',
  'Akadunda kwa grao na kuskia sound ya voice ikimshow, "saul, Saul, niaje unanihanda na persecution

In [ ]:
# save translations in pandas dataframe
import pandas as pd
df = pd.DataFrame(final_translations, columns=['English', 'Sheng'])
# save the dataframe to an CSV file
df.to_csv('dataset.csv', index=False)